<center><h1>Spectral Clustering Algorithm For Community Detection In Graphs</h1></center>

<h3>How to run the algorithm</h3>

By running all the cells, you will get the partitioned graphs and they will be stored in an "output" directory which will be created automatically.

If you want to run the partitioning separately for every graph and not all at once, you can change that in the last cell.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os

import numpy as np
from numpy import linalg as LA
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

import matplotlib.pyplot as plt

from scipy import sparse
from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh

In [2]:
def get_nodes(data_file):
    data = pd.read_csv(data_file, sep = ' ', header = None)
    left_nodes = data[0].values
    right_nodes = data[1].values
    combined_nodes = np.concatenate((left_nodes, right_nodes), axis = 0)
    combined_nodes = np.unique(combined_nodes)
    
    return left_nodes, right_nodes, combined_nodes

In [3]:
def get_unique(arr):
    unique = []
    for i in arr:
        if i not in unique:
            unique.append(i)
    return unique

In [4]:
def get_graph(combined_nodes, left_nodes, right_nodes):
    neighbors = []
    graph = {}
    
    for node in combined_nodes:
        if node in left_nodes:
            indices = np.where(left_nodes == node)
            for index in indices:
                neighbors.append(right_nodes[index])
        if node in right_nodes:
            indices = np.where(right_nodes == node)
            for index in indices:
                neighbors.append(left_nodes[index])

        unique_neighbors = get_unique(neighbors[0])

        unique_neighbors = np.array(unique_neighbors)
        graph[node] = unique_neighbors
        neighbors = []
    
    return graph

In [5]:
def get_adjacency_matrix(graph):
    adjacency_matrix = np.zeros([len(graph), len(graph)])

    for key, value in graph.items():
        for neighbor in value:
            adjacency_matrix[key][value] = 1
    
    return np.array(adjacency_matrix)

In [6]:
def get_eigen(laplacian_matrix, k):
    eigen_values, eigen_vectors = eigsh(laplacian_matrix, k = k, which = 'LA')
    return eigen_values, eigen_vectors

In [7]:
# Use L2 normalization
def normalize_vectors(eigen_vectors):
    eigen_vectors_norm = normalize(
        eigen_vectors, 
        norm = 'l2'
    )
    return eigen_vectors_norm

In [8]:
# Visualize eigenvalues
def visualize_eigenvalues(eigen_values):
    plt.plot(eigen_values)
    plt.title('Eigenvelues')
    plt.show()

In [9]:
# Visualize eigenvectors
def visualize_eigenvectors(eigen_vectors):
    plt.rcParams["figure.figsize"] = (10, 7)

    for i in range(eigen_vectors.shape[1]):
        plt.plot(eigen_vectors[:][i], label= str(i))

    plt.legend(loc='upper right')
    plt.title('Eigenvectors')
    plt.show()

In [10]:
def cluster_graph(k, eigen_vectors):
    kmeans = KMeans(
        n_clusters = k
    )
    
    kmeans.fit(eigen_vectors)
    
    return kmeans

In [11]:
# Visualize cluster distribution
def visualize_class_distribution(k, kmeans):
    distribution = []
    clusters = np.arange(0, k)

    for cluster in range(k):

        elements = np.where(kmeans.labels_ == cluster)[0]
        elements = len(elements)
        distribution.append(elements)

    plt.bar(clusters, distribution)
    plt.xticks(clusters)
    plt.xlabel('number of clusters')
    plt.ylabel('number of elements')
    plt.title('Cluster distribution')
    plt.show()

In [12]:
def save_output(combined_nodes, kmeans, file_name):
    if not os.path.exists('output'):
        os.makedirs('output')
    
    df = pd.DataFrame({'node' : combined_nodes, 'cluster' : kmeans.labels_})
    df = df[['node', 'cluster']]
    df.to_csv(os.path.join('output', file_name.split('.')[0] + '.output'), index = False, sep = ' ')

In [13]:
def main(k, file_name):
    
    data_file = os.path.join('data', file_name)
   
    left_nodes, right_nodes, combined_nodes = get_nodes(data_file)
    
    graph = get_graph(combined_nodes, left_nodes, right_nodes)
    adjacency_matrix = get_adjacency_matrix(graph)

    adjacency_matrix = sparse.csr_matrix(adjacency_matrix)
    laplacian_matrix = csgraph.laplacian(adjacency_matrix, normed=True)
    
    eigen_values, eigen_vectors = get_eigen(laplacian_matrix, k)
    eigen_values = eigen_values.real
    eigen_vectors = eigen_vectors.real
    eigen_vectors = normalize_vectors(eigen_vectors)

    visualize_eigenvalues(eigen_values)
    visualize_eigenvectors(eigen_vectors)
    kmeans = cluster_graph(k, eigen_vectors)
    
    visualize_class_distribution(k, kmeans)
    
    save_output(combined_nodes, kmeans, file_name)

In [15]:
k = [50, 100, 2, 25, 20]
file_array = ['ca-AstroPh.txt', 'ca-CondMat.txt', 'ca-GrQc.txt', 'ca-HepPh.txt', 'ca-HepTh.txt']

for graph in range(len(file_array)):
    main(k[graph], file_array[graph])